# Worksheet 11: Beyond two-group comparisons

In [ ]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(infer))
suppressPackageStartupMessages(library(datateachr))
suppressPackageStartupMessages(library(testthat))
suppressPackageStartupMessages(library(digest))

## Learning Objectives

- Run a simple one-way ANOVA, without knowing the details of the test (the level of detail required is what's presented in the blog post).
- Calculate and interpret prediction intervals.

## Exercise 1: Recap Exercises

### Question 1.1

After how many hypotheses would we expect to make one Type I error, if each hypothesis is tested at a 0.05 significance level? Assuming the data from these tests are independent, that is. Put the number in a variable named `recap1`.

In [ ]:
# recap1 <- FILL_THIS_IN
# your code here
fail() # No Answer - remove if you provide an answer
print(recap1)

In [ ]:
test_that("recap 1", {
  expect_identical(
    digest(as.numeric(recap1)),
    "c72d0198505540505acfb4b2b49911e6"
  )
})
cat("success!")

### Question 1.2

You decide to get some practice with data analysis, so you analyse a different data set each week. For each analysis, you produce a single 90% confidence interval for a parameter. After how many projects would you expect to have made a confidence interval that _does not_ contain the true parameter? Put your answer in a variable named `recap2`.

In [ ]:
# recap2 <- FILL_THIS_IN
# your code here
fail() # No Answer - remove if you provide an answer
print(recap2)

In [ ]:
test_that("recap 2", {
  expect_identical(
    digest(as.numeric(recap2)),
    "2522027d230e3dfe02d8b6eba1fd73e1"
  )
})
cat("success!")

## Exercise 2: One-Way ANOVA

Consider the `attitude` dataset in the datasets R package, which provides the survey scores of six questions (along with an overall score) given to employees of an organization. A tidy version of the data without the overall score is stored below in the `attitudes_tidy` variable.

In [ ]:
attitude_tidy <- attitude %>% 
   select(-rating) %>% 
   pivot_longer(everything(), names_to = "question", values_to = "score")
head(attitude_tidy)

A jitterplot + boxplot of the data is shown below:

In [ ]:
attitude_tidy %>% 
  mutate(question = question %>% 
           as_factor() %>% 
           fct_reorder(score)) %>% 
  ggplot(aes(question, score)) +
  geom_boxplot(width = 0.4, outlier.shape = NA) +
  geom_jitter(width = 0.2, alpha = 1 / 3) +
  theme_minimal()

Although there are visual differences between the groups, perhaps these differences are due to randomness.

For this exercise, denote $\mu_1, \mu_2, \ldots, \mu_6$ the means of each group (ordered as in the above plot).

### Question 2.1

Maybe the mean scores are actually the same for each question. How can we write this as a null hypothesis, using symbols? Place the letter to your answer in a variable named `null_hypothesis`.

**a**\. $\mu_1 = \mu_2 = \cdots = \mu_6$    
**b**\. There are actually 15 null hypotheses, one for each pair of means: $\mu_1 = \mu_2$, $\mu_1 = \mu_3$, $\mu_2 = \mu_3$, etc.     
**c**\. There is an upward trend in score across question type.

In [ ]:
# null_hypothesis <- "FILL_THIS_IN"
# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_that("null hypothesis", {
    hashed_answer <- digest(tolower(null_hypothesis))
    if (hashed_answer == "ddf100612805359cd81fdc5ce3b9fbba") {
        cat("This would result in 15 separate tests -- we're looking for a single test.")
    } else if (hashed_answer == "6e7a8c1c098e8817e3df3fd1b21149d1") {
        cat("The grouping variable has no inherent order, so there's no such thing as a trend.")
    }
    expect_identical(hashed_answer, "127a2ec00989b9f7faf671ed470be7f8")
})
cat("success!")

### Question 2.2

Make a tibble containing the variances for each question. Your tibble should have columns named `"question"`, containing the question, and `"variance"`, containing the variance calculated using the `var()`. There should be six rows -- one for each question. Store the tibble in a variable called `attitude_variances`.

In [ ]:
# attitude_variances <- FILL_THIS_IN
# your code here
fail() # No Answer - remove if you provide an answer
print(attitude_variances)

In [ ]:
test_that("attitude group variances are correct", {
  standardized_answer <- attitude_variances %>% 
    mutate(variance = round(variance, digits = 4)) %>% 
    arrange(question) %>% 
    unclass()
  expect_identical(
    digest(standardized_answer), 
    "f236ecf1d8852cf445dc3e5085d1414f"
  )
})
cat("success!")

### Question 2.3

ANOVA performs more accurately if the variances within each group are the same. Judging by the observed variances, does this look like a decent approximation? Some resources say that, as long as the group with the highest variance isn't many multiples bigger than the smallest group variance, then the specified significance level will be roughly equal to the true significance level. Store either `"a"`, `"b"`, or `"c"` with the variable `variance_call` to indicate your response.

**a**\.  We can assume that the variances are equal.   
**b**\.  The difference in variances between groups is concerning.   
**c**\.  Each group having the same variance appears to be a decent approximation.

In [ ]:
# variance_call <- "FILL_THIS_IN"
# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_that("Variance call is correct", {
  answer_hash <- digest(tolower(variance_call))
  if (answer_hash == "127a2ec00989b9f7faf671ed470be7f8") {
    cat("It's not good language to 'assume' equal variances, ",
        "since the variances are almost surely at least a little different.")
  }
  expect_identical(answer_hash, "6e7a8c1c098e8817e3df3fd1b21149d1")
})
cat("success!")

### Question 2.4

If the null hypothesis is true, then the overall variance could be calculated in two ways:

- The overall variance, as if the grouping variable was absent. Calculate this quantity, and store it in a variable named `variance_overall`
- The average group variance. Calculate this value by averaging the variances within each group. Store this number in a variable named `variance_group_avg`.

Note that there are other ANOVA methods that weigh group variances a different ways, to account for situations where groups have quite different variances.

A test statistic, then, would measure the discrepancy between these two values. Instead of calculating the difference, it works out better to calculate the _ratio_ of the overall variance to the average group variance. A big number (away from 1) is evidence that the null hypothesis may not be true. Store this ratio in a variable named `variance_ratio`.

Framework:

```
variance_overall <- attitude_tidy %>% 
  summarise(FILL_THIS_IN) %>% 
  pull(FILL_THIS_IN)
variance_group_avg <- attitude_variances %>% 
  summarise(FILL_THIS_IN) %>% 
  pull(FILL_THIS_IN)
variance_ratio <- FILL_THIS_IN
```

Your solution:

In [ ]:
# your code here
fail() # No Answer - remove if you provide an answer
print(variance_overall)
print(variance_group_avg)
print(variance_ratio)

In [ ]:
test_that("variance calculations", {
    overall <- digest(round(variance_overall, digits = 2))
    group_avg <- digest(round(variance_group_avg, digits = 2))
    ratio <- digest(round(variance_ratio, digits = 2))
    expect_identical(overall, "6363a1d1005026d0626f68f768ad1ecc")
    expect_identical(group_avg, "f4e70e61a8f13b7409b2b296a1d1d386")
    expect_identical(ratio, "1088ff6cbfbfe0ca38e6d5b79804c749")
})
cat("success!")

### Question 2.5

The _actual_ test statistic -- called the ANOVA F-test statistic -- is a little different from the one we calculated. Its scale is adjusted, and is shifted so that its base is at 0. The interpretation is as a ratio of _treatment variance_ (variability of group means around the overall mean) to the within-group variance. Also, it turns out that this test statistic is well approximated by an _F_-distribution if the null hypothesis is true -- and is exactly true if each group's population is Normally distributed, and the group variances are exactly equal.

Use the `aov()` function to test the hypothesis. The relevant arguments are:

- First argument (`formula`): Takes the form `numeric_column_name ~ group_column_name`.
- `data`: Takes a data frame in tidy format (grouping column and numeric column).

Use the broom package's `tidy()` function to obtain the F-statistic and p-value. Store these values in the variables `f_stat` and `anova_pval`, respectively.

Framework:

```
anova_results <- aov(FILL_THIS_IN ~ FILL_THIS_IN, data = FILL_THIS_IN) %>% 
  broom::tidy()
f_stat <- FILL_THIS_IN
anova_pval <- FILL_THIS_IN
```

In [ ]:
# your code here
fail() # No Answer - remove if you provide an answer
print(anova_results)
print(f_stat)
print(anova_pval)

In [ ]:
test_that("Question 2.6", {
  expect_identical(
    digest(round(f_stat, digits = 4)),
    "201c106c767545f7ae0f82bf91390fe4"
  )
  expect_identical(
    digest(signif(anova_pval, digits = 4)),
    "85048b8d9987b60a8ff053eccf2128b1"
  )
})
cat("success!")

_Note_: You can also run an ANOVA by fitting linear models first, with `lm()`. But, this is a topic for STAT 301! 

### Question 2.6

What is our conclusion under a 0.01 significance level? Assign the letter response to a variable named `anova_conclusion`.

**a**\. Each group has a different / unique mean.   
**b**\. There's not enough evidence to suggest that the group means are different.   
**c**\. The group means are all equal.     
**d**\. At least one group has a mean that is significantly different from the others.   
**e**\. There is a positive linear relationship between score and question.

In [ ]:
# anova_conclusion <- "FILL_THIS_IN"
# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_that("ANOVA conclusion", {
  answer_hash <- digest(tolower(anova_conclusion))
  if (answer_hash == "93a9078c6326f37b481d3e99b60ad987") {
    cat("The 'spacing' and ordering of questions is arbitrary,",
        "so there's no way we can talk about direction and shape",
        "of a relationship.")
  }
  expect_identical(answer_hash, "d110f00cfb1b248e835137025804a23b")
})
cat("success!")

## Exercise 3: Prediction Intervals

Until now, we've been making inference on _population parameters_, such as means, quantiles, variances, etc. As we've seen, this inference allows us to present both (1) our best guess as to what the parameter is (an estimate), along with (2) a description of uncertainty (such as confidence intervals and hypothesis tests). But, 
how can we go about making inference on a _new observation_? 

In the tutorial, you'll explore predictions themselves. In this worksheet, we'll explore the inference part of predictions: _prediction intervals_.

To form a prediction interval, instead of using quantiles of the _(estimated) sampling distribution_, use the _population distribution_. 

Unlike sampling distributions, using the _true_ population distribution to form prediction intervals actually gives us prediction intervals with the specified probability. Using _true_ sampling distributions, on the other hand, is not useful for forming confidence intervals, because we know the true mean is in the center of the sampling distribution, so no interval at all is needed.

Since it's unusual to have access to the population distribution in practice, a common approach is to use the _sample distribution_ when forming prediction intervals. Doing so usually results in the true coverage probability to veer from the specified probability -- something we'll explore a little more in this exercise. 

### Question 3.1

Recall the City of Vancouver building permits dataset `building_permits` in the datateachr R package, which has building permit info on all buildings since 2017. Here is a tibble of the project values of all One-Family Dwellings (having non-zero value):

In [ ]:
project_values <- building_permits %>% 
    filter(specific_use_category == "One-Family Dwelling") %>% 
    select(project_value) %>% 
    filter(project_value > 0)
head(project_values)

<!-- For your own reference, here is a 95% *confidence* interval for the mean (Here, we'll take the population to be infinite: the value in today's dollars of all one-family dwellings since 2017, going on into the future. (Also assuming society remains similar to today's society.) -->
<!-- ) -->

<!-- ```{r} -->
<!-- project_values %>%  -->
<!--   rep_sample_n(size = nrow(.), replace = TRUE, reps = 1000) %>%  -->
<!--   group_by(replicate) %>%  -->
<!--   summarise(mean_value = mean(project_value), .groups = "drop") %>%  -->
<!--   pull(mean_value) %>%  -->
<!--   quantile(c(0.025, 0.975)) %>%  -->
<!--   unname() -->
<!-- ``` -->

If a new one-family dwelling is going to be built, what is a 90% prediction interval for this property's value (given that it's above 0)? Store this interval as a vector of length two (lower first, upper second), as a variable named `dwelling_pi`. 

Technicalities:

- Use the `quantile()` function (and its default `type = 7` argument).
- Ensure equal probability on both sides of the distribution. 

In [ ]:
# dwelling_pi <- FILL_THIS_IN
# your code here
fail() # No Answer - remove if you provide an answer
print(dwelling_pi)

In [ ]:
test_that("One-Family Dwelling PI", {
  expect_identical(
    digest(round(unname(dwelling_pi), digits = 2)),
    "ebbcb0aa1dd53b57e2da4ca96cfd46d3"
  )
})
cat("success!")

### Question 3.2

Based on the definitions of prediction interval and confidence interval, what happens to these intervals as the sample size increases? Place the letters of your response in alphabetical order as a string, in a variable named `interval_trends`.

**a**\. Confidence intervals remain roughly the same width, because the confidence level remains the same.    
**b**\. Confidence intervals get narrower as we zero in on the true parameter.    
**c**\. Confidence intervals expand as we gain more and more confidence with larger sample sizes.   
**d**\. Prediction intervals remain roughly the same width -- after all, we're just getting better and better at estimating the true quantiles making up the boundaries of the prediction interval.   
**e**\. Prediction intervals get narrower as we zero in on the true future value.    
**f**\. Prediction intervals expand as we gain more and more confidence with larger sample sizes. 

In [ ]:
# interval_trends <- "FILL_THIS_IN"
# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_that("Realities of smaller coverage probability", {
  answer_hash <- digest(tolower(interval_trends))
  expect_identical(answer_hash, "95767987b2037a2f09c4e5c0997ec206")
})
cat("success!")

### Question 3.3

To build concepts, let's interpret our sample as being the entire population. What proportion of this population's property values fall in the prediction interval? This is called the _true coverage probability_, because it's the proportion of the population that truly falls in the interval. Take the interval to be inclusive of both endpoints. Store the proportion in a variable named `dwellings_in_pi`. 

(By the way: the reason you don't get 90% exactly is because the population is finite. This means that changing one of the endpoints of your prediction interval to the next higher or lower value will cause the confidence level to also make a discrete jump to another value. This is especially characteristic of discrete data.)

Framework:

```
dwellings_in_pi <- project_values %>% 
  summarise(prop = mean(
    FILL_THIS_IN >= FILL_THIS_IN &
      FILL_THIS_IN <= FILL_THIS_IN
  )) %>% 
  pull(FILL_THIS_IN)
```

Your solution:

In [ ]:
# dwellings_in_pi <- FILL_THIS_IN
# your code here
fail() # No Answer - remove if you provide an answer
print(dwellings_in_pi)

In [ ]:
test_that("Proportion in prediction interval", {
  expect_identical(
    digest(round(dwellings_in_pi, digits = 4)),
    "e2ad489efa88de3347f33df53dca0e44"
  )
})
cat("success!")

### Question 3.4

Let's still interpret our sample as being the population, so that we can see an example of the specified coverage probability veering off from the true coverage probability. 

To see how the true coverage probabilities vary depending on the sample, let's take multiple (1000) samples, and calculate the true coverage probability for each of them. Remember, since our original sample is the entire population, it wouldn't make sense to sample with replacement, because we could potentially duplicate members of our population. Use a seed of `123`, and store your final vector of 1000 coverage probabilities in a vector named `coverage`. 

Scaffolding:

```
set.seed(123)
## Draw 1000 samples of size 100:
sample_pi <- project_values %>% 
  rep_sample_n(FILL_THIS_IN, replace = FILL_THIS_IN, reps = FILL_THIS_IN) %>% 
  group_by(replicate) %>% 
  summarise(lower = quantile(FILL_THIS_IN, FILL_THIS_IN),
            upper = quantile(FILL_THIS_IN, FILL_THIS_IN))
## Get vector of project values:
project_values_vec <- project_values$project_value
## Calculate coverage probabilities:
coverage <- sample_pi %>% 
  group_by(replicate) %>% 
  summarise(coverage = mean(
    project_values_vec >= FILL_THIS_IN &
      FILL_THIS_IN
  )) %>% 
  pull(coverage)
```

Your solution:

In [ ]:
# set.seed(123)
# FILL_THIS_IN
# your code here
fail() # No Answer - remove if you provide an answer
head(coverage)

In [ ]:
test_that("Sample coverages", {
  expect_identical(
    digest(round(unname(sort(coverage)), digits = 4)),
    "a87fa2047aca2dd118f118b0b49dd3e3"
  )
})
cat("success!")

### Question 3.5

Here is a histogram of the coverage probabilities you produced, with a vertical line representing the intended coverage probability:

In [ ]:
qplot(coverage, bins = 30) +
  theme_minimal() +
  xlab("Coverage probability") +
  ylab("Count") +
  geom_vline(xintercept = 0.9)

As you can see, the true coverage probability depends on the sample drawn. This histogram shape is typical, with more mass on the left, suggesting that smaller coverage probabilities are more likely.

Based on your simulation, just how likely are you to get a coverage probability less than the specified coverage probability of 0.9? Put this probability in a variable named `prob_smaller`.

In [ ]:
# prob_smaller <- FILL_THIS_IN
# your code here
fail() # No Answer - remove if you provide an answer
print(prob_smaller)

In [ ]:
test_that("Probability of smaller coverage", {
  expect_identical(
    digest(round(prob_smaller, digits = 3)),
    "4b2646cfd98fd99bc210dbf47d832ffd"
  )
})
cat("success!")

### Question 3.6

Now, consider our original population of one-family dwellings now and in the future. If it turns out that the true coverage probability of our prediction interval is smaller than the specified probability of 90%, what does this mean? Keep in mind that we can't know how the true coverage probability will compare. Place your selection(s) in alphabetical order in one string in a variable named `realities`.

**a**\. More one-family dwellings being built will have property values outside of our interval than we think.   
**b**\. Less one-family dwellings being built will have property values outside of our interval than we think.   
**c**\. In order to capture the true coverage probability, we would need to expand our prediction interval.   
**d**\. In order to capture the true coverage probability, we would need to shrink our prediction interval.   

Before we end this section, it's worth saying that there's still merit in using a sample-based prediction interval, even though the true coverage probability might be different from the specified one. Just as we have to accept error when estimating a parameter like the mean, we have to accept some level of error here, too. That being said, there *are* ways of adjusting prediction intervals to more closely match the true coverage probability. A famous one is when the population distribution is Gaussian (just google "prediction interval", and many resources will show a formula based on a Normal population). But, we won't cover these methods in this course. 

In [ ]:
# realities <- "FILL_THIS_IN"
# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_that("Realities of smaller coverage probability", {
  answer_hash <- digest(tolower(realities))
  expect_identical(answer_hash, "7ecaefcc6ebc9848e7cb04b5c783ae0a")
})
cat("success!")